In [1]:
# Импорт нужных библиотек и реализация вспомогательных функций

import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
pd.options.mode.chained_assignment = None
import re

GET_INN_URL = 'https://yandex.ru/search/?text='
EGRUL_URL = 'https://egrul.itsoft.ru/'
REPUTATION_URL = 'https://vbankcenter.ru/contragent/'


def get_company_list_by_product_metalloprokat(product_name):
    # парсер www.metalloprokat.ru
    resut = json.dumps({})

    TARGET_URL = 'https://www.metalloprokat.ru/company/?q='

    site_raw = requests.get(TARGET_URL+product_name).text
    parsed = bs(site_raw, features='lxml')
    comp_count = int(parsed.find_all('div', {'id': 'tabs'})[
        0].find_all('span', {'class': 'count'})[1].text)

    if comp_count < 1:
        return resut

    companies = parsed.find_all('div', {'class': 'title'})

    sellers = dict()

    for company in companies:
        try:
            seller_name = company.find_all('a')[0].text[1:-1]
            sellers[seller_name] = []
            sellers[seller_name].append(company.find_all('a')[0]['href'])
        except:
            pass

    reviews = parsed.find_all(
        'li', {'class': 'links_comment item medium float-left clearfix'})
    for review in reviews:
        try:
            print(review.a['data-href'])
        except:
            pass

    for seller in sellers:
        try:
            raw_for_inn = requests.get(GET_INN_URL+seller+'+ИНН').text
            parsed = bs(raw_for_inn, features='lxml')
            inn = parsed.find_all('div', {'class': 'serp-list serp-list_right_yes serp-list_complementary_yes'})[
                0].find_all('div', {'class': 'KeyValue-ItemValue'})[0].text
            sellers[seller].append(inn)
            sellers[seller].append(json.loads(
                requests.get(EGRUL_URL+inn+'.json').text))
        except:
            pass

    resut = json.dumps(sellers)

    return resut


def get_company_list_by_product(product_name):
    # базовый метод для парсинга в который добавляем спец парсеры
    json_res = get_company_list_by_product_metalloprokat(product_name)
    with open(product_name+'.txt', 'w') as rfile:
        rfile.write(json_res)
    pass


In [2]:
# Для ручного ввода запроса
#req_str = input('Enter request:')
# get_company_list_by_product(req_str)

**_Проверка методики разложения наименования на группы_**


In [3]:
# Загрузка тестовых данных
input_data = pd.read_excel('task3/poiskpostav_v1.xlsx')

In [4]:
# Подготовка шаблонов для разбора строк

# Шаблон для "стандартов"
patGOST = re.compile(
    r'(ГОСТ\s?Р?\s?(?:ИСО)?\s?\d*[\/]?\d*\s?(?:DIN)?\s?\d*[\/]?\d*)')
# Шаблон для метрических параметров
patMetricSize = re.compile(r'[мМ]\d{1,}[хХ]?\d{1,}')
# Шаблон для текстового описания
patWords = re.compile(r'((?:[A-ZА-Я]{1,})?[а-яa-z]{2,})')
patNotText = re.compile(r'!(\s?((?:[A-ZА-Я]{1})?[а-яa-z]{2,})\s?)')


In [5]:
def compileFromValues(values):
    result = None
    if values.lastindex > 0:
        prev = ''
        if result:
            prev = result+' '
        result = prev + values[0]
    return result

def clearFromNone(value):
    return value[5:]


input_data['Стандарт'] = None

# заполняем поле "Стандарт"
for i in range(len(input_data)):
    gosts = re.finditer(patGOST, input_data['Наименование'][i])
    for gost in gosts:
        input_data['Стандарт'][i] = str(input_data['Стандарт'][i]) +', '+compileFromValues(gost)
        input_data['Стандарт'][i] = clearFromNone(input_data['Стандарт'][i])

        '''if gost.lastindex > 0:
            prev = ''

            if input_data['Стандарт'][i]:
                prev = input_data['Стандарт'].loc[i]+', '
            input_data['Стандарт'][i] = prev + gost[0]'''

# заполняем поле "Чистое наименование"
input_data['Чистое наименование'] = None
input_data['Характеристики'] = None
for i in range(len(input_data)):
    # обрабатываем первый тип: "Наименование написано вот так"
    words = re.finditer(patWords, str(input_data['Наименование'][i]).replace(
        input_data['Стандарт'][i], ""))
    """for word in words:
        word"""
    # собираем строку
    for word in words:
        input_data['Чистое наименование'][i] = str(
            input_data['Чистое наименование'][i])+" " + compileFromValues(word)
    # если строка собралась, нормализируем её, удаляя лишний "мусор"
    if input_data['Чистое наименование'][i]:
        input_data['Чистое наименование'][i] = clearFromNone(input_data['Чистое наименование'][i])

        input_data['Характеристики'][i] = str(input_data['Наименование'][i][1:]).replace(
            input_data['Стандарт'][i], "").replace(input_data['Чистое наименование'][i], "")

    else:
        raw_str = str(input_data['Наименование'][i]).replace(
            input_data['Стандарт'][i], "")
        splited = raw_str.split(";")
        input_data['Чистое наименование'][i] = splited[0][2:]


input_data


,№,Наименование,Стандарт,Чистое наименование,Характеристики
0,1,* Манжета М50х70 ГОСТ 22704,ГОСТ 22704,Манжета,М50х70
1,2,* Манжета М65х90 ГОСТ 22704,ГОСТ 22704,Манжета,М65х90
2,3,* Манжета М220х250 ГОСТ 22704,ГОСТ 22704,Манжета,М220х250
3,4,* Манжета М60х80 ГОСТ 22704,ГОСТ 22704,Манжета,М60х80
4,5,* Манжета резиновая армированная для валов 1.2...,ГОСТ 8752,Манжета резиновая армированная для валов,1.2-120х150х12-1
5,6,* Пластина техническая 2Н-I-ТМКЩ-С-4 ГОСТ 7338,ГОСТ 7338,Пластина техническая,2Н-I-ТМКЩ-С-4
6,7,"* Рукав всасывающий В-1-75-У рабочий вакуум 0,...",ГОСТ 5398,Рукав всасывающий рабочий вакуум,"Рукав всасывающий В-1-75-У рабочий вакуум 0,0..."
7,8,* Рукав с текстильным каркасом В(II)-16-25-38-...,ГОСТ 18698,Рукав текстильным каркасом,Рукав с текстильным каркасом В(II)-16-25-38-ХЛ
8,9,* Пластина техническая 1Н-I-ТМКЩ-С-5 ГОСТ 7338,ГОСТ 7338,Пластина техническая,1Н-I-ТМКЩ-С-5
9,10,* Манжета резиновая армированная для валов 1.2...,ГОСТ 8752,Манжета резиновая армированная для валов,1.2-90х120х12-1


In [6]:
# Парсинг Яндекса по номенклатуре

def getTopFromYandex(request_str):
    # Получаем наменклатурную позицию и возвращаем лист пар: URL = описание
    result_url = []
    result_description = []
    result = []
    SEARCH_URL = "https://yandex.ru/search/direct?text="
    SEARCH_URL_POSTFIX = "&filters_docs=direct_cm"
    request_str = "купить "+request_str[1:]

    raw_result = requests.get(SEARCH_URL+request_str+SEARCH_URL_POSTFIX).text

    parsed_result = bs(raw_result, features='lxml')

    cards = parsed_result.find_all('li', {'class': "serp-item desktop-card"})

    for card in cards:
        links = card.find_all('a', {'target': "_blank"})
        for link in links:
            result_url.append(link.get('href'))
        descriptions = card.find_all('span', {'role': 'text'})

        for description in descriptions:
            result_description.append(description.text)

    for _ in range(min(len(result_url), len(result_description))):
        result.append([result_url[_], result_description[_]])

    return result


In [8]:
# Пример парсинга имеющейся номенклатуры
# результат выгружается в джейсон curdump.txt
# просмотреть содержимое можно здесь -> http://jsonviewer.stack.hu

res = dict()
for i in range(len(input_data)):
    res[input_data['Наименование'][i]] = getTopFromYandex(
        input_data['Наименование'][i])

res_json = json.dumps(res)
with open('curdump.txt', 'w') as res_file:
    res_file.write(res_json)
